In [1]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
diabetes_130_us_hospitals_for_years_1999_2008 = fetch_ucirepo(id=296) 
  
# data (as pandas dataframes) 
X = diabetes_130_us_hospitals_for_years_1999_2008.data.features 
y = diabetes_130_us_hospitals_for_years_1999_2008.data.targets 
  
# metadata 
print(diabetes_130_us_hospitals_for_years_1999_2008.metadata) 
  
# variable information 
print(diabetes_130_us_hospitals_for_years_1999_2008.variables) 


C:\Users\Nishant\AppData\Roaming\Python\Python313\site-packages\ucimlrepo\fetch.py:97: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


{'uci_id': 296, 'name': 'Diabetes 130-US Hospitals for Years 1999-2008', 'repository_url': 'https://archive.ics.uci.edu/dataset/296/diabetes+130-us+hospitals+for+years+1999-2008', 'data_url': 'https://archive.ics.uci.edu/static/public/296/data.csv', 'abstract': 'The dataset represents ten years (1999-2008) of clinical care at 130 US hospitals and integrated delivery networks. Each row concerns hospital records of patients diagnosed with diabetes, who underwent laboratory, medications, and stayed up to 14 days. The goal is to determine the early readmission of the patient within 30 days of discharge.\nThe problem is important for the following reasons. Despite high-quality evidence showing improved clinical outcomes for diabetic patients who receive various preventive and therapeutic interventions, many patients do not receive them. This can be partially attributed to arbitrary diabetes management in hospital environments, which fail to attend to glycemic control. Failure to provide pro

In [ ]:
X.info()


In [ ]:
X.isnull()

In [ ]:
X.isnull().sum()

In [ ]:
import numpy as np
import pandas as pd
X.fillna("NAN")

In [ ]:
X.isnull().sum()

In [ ]:
X.fillna(0, inplace=False)


In [ ]:
X.isnull().sum()

In [ ]:
X.drop('weight',axis=1).head()

In [ ]:
X.columns


In [ ]:
X.columns

In [ ]:
X.columns=X.columns.str.lower()
X.columns=X.columns.str.replace(' ','_')

In [ ]:
X.columns

In [ ]:
y.isnull().sum().head()

In [ ]:
y.head()

In [ ]:
mapping = {'<30': 1, '>30': 0, 'NO': 0}
y['readmission_binary'] = y['readmitted'].map(mapping)

In [ ]:
y.head(20)

In [ ]:
X['time_in_hospital'] = X['time_in_hospital'].replace(0, 1)
X['acuity_score'] = (X['num_lab_procedures'] + 
                    X['num_medications'] + 
                      X['number_diagnoses']) /X['time_in_hospital']
X['acuity_score'] =X['acuity_score'].round(2)

In [ ]:
print(X[['payer_code', 'acuity_score', 'time_in_hospital']].sort_values(by='acuity_score', ascending=False).head())

In [ ]:
X.head()

In [ ]:

AVG_READMISSION_COST = 13500
X['historical_loss'] = y['readmission_binary'] * AVG_READMISSION_COST
dept_loss = X.groupby('medical_specialty')['historical_loss'].sum().sort_values(ascending=False).head(10)

print("Top 10 Cost-Draining Departments:")
print(dept_loss)

In [ ]:
X.head()

In [ ]:
import numpy as np


def calculate_nurse_demand(acuity):
    if acuity > 7:
        return 1/3  
    else:
        return 1/5  


X['nurse_required_count'] = X['acuity_score'].apply(calculate_nurse_demand)


staffing_needs = X.groupby('medical_specialty').agg({
    'payer_code': 'count',
    'nurse_required_count': 'sum',
    'acuity_score': 'mean'
}).rename(columns={'payer_code': 'Total_Patients', 'nurse_required_count': 'Nurses_Needed'})


staffing_needs['Nurses_Needed'] = np.ceil(staffing_needs['Nurses_Needed']).astype(int)

print(staffing_needs.sort_values(by='Nurses_Needed', ascending=False).head(10))

In [ ]:
X.head()

In [ ]:
# Use Parquet for faster loading and smaller file size

# Or standard CSV
X.to_csv('hospital_data_cleaned.csv', index=False)